In [1]:
from abacusnbody.data.compaso_halo_catalog import CompaSOHaloCatalog
from hmf import MassFunction
from astropy.cosmology import Planck18
import matplotlib.pyplot as plt  
import numpy as np

In [8]:
global type, cosmo, IC, redshift, mass, volume, startfile, endfile
type = 'small'
cosmo = 'c000'
IC = 'ph3000'
redshift = '3.000'
print(redshift)
mass = 2*10**9
volume = 500**3
startfile = 0
endfile = 0

3.000


In [10]:
# Mass function from hmfcalc
def hmfcalc():
   global redshift
   redshift = float(redshift)
   global hmf
   hmf = MassFunction(cosmo_model=Planck18, z=redshift)

In [20]:
# Check for corrupt files
def filecheck():
   global redshift
   redshift = float(redshift)
   a = []
   for i in range(start,end): 
      if i < 10:
          a.append(str(path + redshift + '/halo_info/halo_info_00' + str(i) + '.asdf'))
      else:   
          a.append(str(path + redshift + '/halo_info/halo_info_0' + str(i) + '.asdf'))
   print (a)

   cat = CompaSOHaloCatalog(a)

In [4]:
#Calculate volume of data used
def newvloume():
   global newvolume
   newcvolume = volume*((endfile-startfile)/33)
   

In [5]:
# Load files and calculate halo mass together, here all files are loaded at once and mass calulated together
def loadfilesall():
   print("redshift =",redshift)
   print("Status: Reading the files")
   cat = CompaSOHaloCatalog(path + redshift + '/halo_info/',cleaned=False)

   total_num_halos = len(cat.halos)
   print("Number of halos =", total_num_halos)

   print("Status: Calculating the mass of halos")
   
   num = np.array(cat.halos['N'])
   global halo_mass
   halo_mass = []
   for i in range (total_num_halos):
      halo_mass.append(num[i]*mass)

In [6]:
def onebyone():
   print("redshift =",redshift)
   print("Status: Reading the files")
   
   global halo_mass
   halo_mass = []
   for i in range(startfile,endfile+1): 
      print("Procesing file", i)
      if i < 10:
         file = str('AbacusSummit Public Data Access/AbacusSummit_' + type + '_' + cosmo +'_' + IC + '/halos/z' + redshift + '/halo_info/halo_info_00' + str(i) + '.asdf')
      else:   
         file = str('AbacusSummit Public Data Access/AbacusSummit_' + type + '_' + cosmo +'_' + IC + '/halos/z' + redshift +  '/halo_info/halo_info_0' + str(i) + '.asdf')
      cat = CompaSOHaloCatalog(file, cleaned=False)
      num_halos = len(cat.halos)
      num = np.array(cat.halos['N'])
      for i in range (num_halos):
         halo_mass.append(num[i]*mass)
      del cat
   
   total_num_halos = len(halo_mass)
   print("Number of halos =", total_num_halos)


In [15]:
# Calculating hmf dndlnm
def plt_hmf_dndlnm():
    print("Status: Calculating height and with of mass histogarm")
    hght, binedge = np.histogram(halo_mass, bins=np.logspace(np.log10(0.5*10**11),np.log10(0.5*10**13), 50))

    print("Status: Calculating the mass function")
    bin_centers = []
    for i in range (len(binedge)-1):
        bin_centers.append(np.sqrt(binedge[i+1]*binedge[i]))

    binsize = []
    for i in range (len(binedge)-1):
        binsize.append(binedge[i+1]-binedge[i])

    newhght = []
    for i in range(len(hght)):              
        newhght.append(hght[i]/((binsize[i]/bin_centers[i])*volume))

    #error=np.sqrt(newhght)
    #hght = hght/binedge
    
    plt.figure(figsize=(10,10))
    plt.scatter(bin_centers,newhght,s=10)
    plt.loglog(hmf.m,hmf.dndlnm)
    #plt.errorbar(bin_centers, newhght, yerr=error)
    plt.title(redshift +' dn_Vdlnm')
    plt.xlabel("$m$")
    plt.ylabel("$\dfrac{dn}{Vdlnm}$")
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim((0.5*10**11,10**13))
    plt.ylim((0.5*10**-11,10**-2))
    plt.savefig(redshift +'_dn_Vdlnm_'+ str(startfile) + '_' + str(endfile) + "_hmf.jpg")

3.0


In [ ]:
f, (a0, a1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [2, 1]},figsize=(20,20))
# Calculating hmf dndlnm and error
def plt_hmf_dndlnm():
   print("Status: Calculating height and with of mass histogarm")
   hght, binedge = np.histogram(halo_mass, bins=np.logspace(np.log10(0.5*10**11),np.log10(0.5*10**13), 50))

   print("Status: Calculating the mass function")
   bin_centers = []
   for i in range (len(binedge)-1):
      bin_centers.append(np.sqrt(binedge[i+1]*binedge[i]))

   binsize = []
   for i in range (len(binedge)-1):
      binsize.append(binedge[i+1]-binedge[i])

   newhght = []
   for i in range(len(hght)):              
      newhght.append(hght[i]/((binsize[i]/bin_centers[i])*volume))

   newhmf = np.interp(bin_centers,hmf.m,hmf.dndlnm)
   error = ((newhmf-newhght)/newhmf)*100
   print (error)

   #f.tight_layout()

   #fig, ax = plt.subplots(figsize=(15,10))
   #a0 = plt.subplot2grid((3,3), (0, 0))
   #a1 = plt.subplot2grid((3,3), (1, 0))

   a0.scatter(bin_centers,newhght,s=10)
   a0.loglog(hmf.m,hmf.dndlnm)

   a0.set_title(redshift + ' ' + type + ' ' + cosmo + ' ' + IC +'_dn_Vdlnm & Error')
   a0.set_xlabel("Mass")
   a0.set_ylabel("$\dfrac{dn}{Vdlnm}$")
   a0.set_xscale('log')
   a0.set_yscale('log')
   a0.set_xlim((0.5*10**11,10**13))
   a0.set_ylim((0.5*10**-6,10**-1))


   a1.plot(bin_centers,error)
   a1.set_xlabel("Mass")
   a1.set_xscale('log')
   a1.set_ylabel("Percentage Error")
   a1.set_ylim((-100,100))

In [11]:
# Calculating hmf dndm
def plt_hmf_dndm():
   print("Status: Calculating height and with of mass histogarm")
   hght, binedge = np.histogram(halo_mass, bins=np.logspace(np.log10(0.5*10**11),np.log10(0.5*10**13), 50))

   print("Status: Calculating the mass function")
   bin_centers = []
   for i in range (len(binedge)-1):
      bin_centers.append(np.sqrt(binedge[i+1]*binedge[i]))
   
   binsize = []
   for i in range (len(binedge)-1):
      binsize.append(binedge[i+1]-binedge[i])
   
   newhght = []
   for i in range(len(hght)):              
      newhght.append(hght[i]/(binsize[i]*volume))
   #error=np.sqrt(newhght)
   #hght = hght/binedge
   
   plt.figure(figsize=(10,10))
   plt.scatter(bin_centers,newhght,s=10)
   plt.loglog(hmf.m,hmf.dndm)
   #plt.errorbar(bin_centers, newhght, yerr=error)
   plt.title(redshift +' dn_Vdm')
   plt.xlabel("$m$")
   plt.ylabel('$\dfrac{dn}{Vdm}$')
   plt.xscale('log')
   plt.yscale('log')
   plt.xlim((0.5*10**11,10**13))
   plt.ylim((0.5*10**-22,10**-11))
   plt.savefig(redshift +'_dn_Vdm_'+ str(startfile) + '_' + str(endfile) + "_hmf.jpg")

In [ ]:
#Error
def error()
    print(len(hmf.m))
    print(len(hmf.dndlnm))
    newhmf = np.interp(bin_centers,hmf.m,hmf.dndlnm)
    print(newhmf)
    print(len(bin_centers))

    error = []
    for i in range(len(bin_centers)):
        error.append(((newhmf-newhght)*100)/newhmf)
    print(len(error))
    print(newhght)
    print(len(error))
    plt.plot(bin_centers, error)